In [11]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

2024-10-07 07:35:18.772159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 07:35:18.795190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 07:35:18.801769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 07:35:18.937223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-07 07:35:20.324814: W tensorflow/compiler/tf2

In [13]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1728279323.172764    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728279323.604566    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728279323.604786    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [14]:
BATCH_SIZE=32

#### Train/test sets

In [15]:
# Train set : CIFAKE/train
# Class FAKE : CIFAKE/train/FAKE
# Class REAL : CIFAKE/train/REAL
train_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/train', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

# Test set : CIFAKE > test
# Class FAKE : CIFAKE/test/FAKE
# Class REAL : CIFAKE/test/REAL
validation_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/test', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

Found 100000 files belonging to 2 classes.


I0000 00:00:1728279327.394588    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728279327.394796    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728279327.394952    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728279327.455541    7112 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 20000 files belonging to 2 classes.


In [16]:
class_names = train_set.class_names
print(f'Training classes: {class_names}')

class_names = validation_set.class_names
print(f'Validation classes: {class_names}')

Training classes: ['FAKE', 'REAL']
Validation classes: ['FAKE', 'REAL']


#### EDA

In [17]:
import glob
print('Class distribution: ')
print( f'Train REAL images: {len(glob.glob('CIFAKE/train/REAL/*'))}'  )
print( f'Train FAKE images: {len(glob.glob('CIFAKE/train/FAKE/*'))}'  )

print( f'Test REAL images: {len(glob.glob('CIFAKE/test/REAL/*'))}'  )
print( f'Test FAKE images: {len(glob.glob('CIFAKE/test/FAKE/*'))}'  )

Class distribution: 
Train REAL images: 50000
Train FAKE images: 50000
Test REAL images: 10000
Test FAKE images: 10000


In [18]:
# Show random image

#### Image normalisation

In [19]:
# Image normalisation

#### Build model

In [20]:
# Build model
def CNN_model01():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),

        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer=Adam(learning_rate=0.01), 
                  loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()],                
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [21]:
CNN_model01().summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │       172,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 173,745 (678.69 KB)

 Trainable params: 173,745 (678.69 KB)

 Non-trainable params: 0 (0.00 B)

#### Train model

In [22]:
N_EPOCHS = 50

In [23]:
time_start = dt.now()
print(f'Start training, time: {time_start.time()}')

model = CNN_model01()

history = model.fit(
  train_set,
  validation_data=validation_set,
  epochs=N_EPOCHS,
  verbose=1
)

print(f'Time elapsed: {dt.now() - time_start}')

Start training, time: 07:35:29.567310
Epoch 1/50


I0000 00:00:1728279330.275825    9621 service.cc:146] XLA service 0x79d898002140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728279330.275841    9621 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2024-10-07 07:35:30.313303: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-07 07:35:30.475254: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  22/3125 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - accuracy: 0.5308 - loss: 2.4833 - precision_1: 0.5296 - recall_1: 0.6214

I0000 00:00:1728279331.821558    9621 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.7155 - loss: 0.5893 - precision_1: 0.6979 - recall_1: 0.7592 - val_accuracy: 0.7854 - val_loss: 0.4715 - val_precision_1: 0.7439 - val_recall_1: 0.8704
Epoch 2/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7820 - loss: 0.4748 - precision_1: 0.7706 - recall_1: 0.8019 - val_accuracy: 0.7940 - val_loss: 0.4549 - val_precision_1: 0.7793 - val_recall_1: 0.8204
Epoch 3/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7911 - loss: 0.4634 - precision_1: 0.7792 - recall_1: 0.8115 - val_accuracy: 0.7987 - val_loss: 0.4484 - val_precision_1: 0.7924 - val_recall_1: 0.8093
Epoch 4/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7923 - loss: 0.4588 - precision_1: 0.7747 - recall_1: 0.8230 - val_accuracy: 0.7967 - val_loss: 0.4504 - val_precision_1: 0.7629 - val_recall_1: 0.8610
Epoch 5/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7941 - loss: 0.4549 - precision_1: 0.7748 - recall_1: 0.8278 -

In [24]:
# X_train, y_train = train_set[]

# from sklearn.model_selection import cross_val_score

# k_fold_acc = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')

# # NOTE - Try with gridsearch just for cross validation

##### GridSearch

In [28]:
from sklearn.model_selection import GridSearchCV
# Train with GRID search
param_grid = dict(
    epochs=[25],
    batch_size=[32],
    optimizer=['adam'],
)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=10, scoring='accuracy')

In [30]:
from datetime import datetime as dt

start_t = dt.now()
print(f'GridSearch start time: {start_t.hour}:{start_t.minute:02}:{start_t.second:02}')
grid_result = grid.fit(train_set, validation_data=(train_set))

print(f'GridSearch complete. Time elapsed: {dt.now()-start_t}')

GridSearch start time: 7:54:10


TypeError: Singleton array array(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
      dtype=object) cannot be considered a valid collection.

#### Visualisation

In [25]:
cols = pd.DataFrame(history.history).columns
cols

Index(['accuracy', 'loss', 'precision_1', 'recall_1', 'val_accuracy',
       'val_loss', 'val_precision_1', 'val_recall_1'],
      dtype='object')

In [26]:
hist = history.history

n = 5
train_col = cols[2]
val_col = f'val_{train_col}'

fig = make_subplots(rows=2, cols=2, subplot_titles=("Loss over epochs", f"{train_col} over epochs"), vertical_spacing=0.07)

# Loss
fig.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines+markers', name='Training Loss'), row=1, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=1 )

# 
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=1, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=1, col=2)

#
train_col = cols[3]
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=1)

train_col = 'accuracy'
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=2)


fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=1, col=2)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=2)

fig.update_layout(
    # title_text="Training and validation metrics over epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1400, height=800
)

for annotation in fig['layout']['annotations']:
    annotation['y'] = annotation['y'] + 0.002

fig

In [27]:
# Confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_classes)

#### Class activation map